In [ ]:
# Notebook: text_processing.ipynb

import os
import pathlib
import json
from typing import List, Dict, Any
import logging

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Importar LangChain para el procesamiento de texto
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    LANGCHAIN_AVAILABLE = True
except ImportError:
    LANGCHAIN_AVAILABLE = False
    logger.warning("LangChain no está instalado. Por favor, instala langchain.")


def load_text_files(input_dir: str) -> Dict[str, str]:
    """
    Carga todos los archivos de texto de un directorio.
    
    Args:
        input_dir: Directorio con archivos de texto
        
    Returns:
        Diccionario con nombres de archivo como claves y contenido como valores
    """
    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"El directorio {input_dir} no existe")
    
    text_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.txt')]
    logger.info(f"Se encontraron {len(text_files)} archivos de texto en {input_dir}")
    
    results = {}
    
    for text_file in text_files:
        text_path = os.path.join(input_dir, text_file)
        try:
            with open(text_path, 'r', encoding='utf-8') as f:
                content = f.read()
            results[text_file] = content
        except Exception as e:
            logger.error(f"Error al leer {text_file}: {e}")
    
    return results


def split_text_into_chunks(text: str, chunk_size: int = 1000, chunk_overlap: int = 200) -> List[str]:
    """
    Divide un texto en chunks más pequeños usando LangChain.
    
    Args:
        text: Texto a dividir
        chunk_size: Tamaño aproximado de cada chunk
        chunk_overlap: Superposición entre chunks consecutivos
        
    Returns:
        Lista de chunks de texto
    """
    if not LANGCHAIN_AVAILABLE:
        raise ImportError("LangChain es necesario para dividir el texto. Por favor, instala langchain.")
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    chunks = text_splitter.split_text(text)
    return chunks


def process_all_texts(texts: Dict[str, str], chunk_size: int = 1000, chunk_overlap: int = 200) -> Dict[str, List[Dict[str, Any]]]:
    """
    Procesa todos los textos y los divide en chunks.
    
    Args:
        texts: Diccionario con nombres de archivo como claves y contenido como valores
        chunk_size: Tamaño aproximado de cada chunk
        chunk_overlap: Superposición entre chunks consecutivos
        
    Returns:
        Diccionario con nombres de archivo como claves y listas de chunks como valores
    """
    results = {}
    
    for filename, text in texts.items():
        logger.info(f"Procesando {filename}...")
        
        # Extraer nombre base sin extensión
        base_name = os.path.splitext(filename)[0]
        
        # Dividir en chunks
        try:
            chunks = split_text_into_chunks(text, chunk_size, chunk_overlap)
            
            # Crear metadatos para cada chunk
            processed_chunks = []
            for i, chunk in enumerate(chunks):
                processed_chunks.append({
                    "text": chunk,
                    "metadata": {
                        "source": filename,
                        "chunk_id": i,
                        "document": base_name
                    }
                })
            
            results[filename] = processed_chunks
            logger.info(f"Se generaron {len(chunks)} chunks para {filename}")
            
        except Exception as e:
            logger.error(f"Error al procesar {filename}: {e}")
    
    return results


def save_chunks(chunks_dict: Dict[str, List[Dict[str, Any]]], output_dir: str) -> None:
    """
    Guarda los chunks procesados en archivos JSON.
    
    Args:
        chunks_dict: Diccionario con nombres de archivo como claves y listas de chunks como valores
        output_dir: Directorio donde guardar los chunks
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Guardar cada documento en su propio archivo
    for filename, chunks in chunks_dict.items():
        base_name = os.path.splitext(filename)[0]
        output_path = os.path.join(output_dir, f"{base_name}_chunks.json")
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(chunks, f, ensure_ascii=False, indent=2)
        
        logger.info(f"Chunks guardados en {output_path}")
    
    # Guardar todos los chunks en un solo archivo (útil para cargar todo de una vez)
    all_chunks = []
    for chunks_list in chunks_dict.values():
        all_chunks.extend(chunks_list)
    
    all_chunks_path = os.path.join(output_dir, "all_chunks.json")
    with open(all_chunks_path, 'w', encoding='utf-8') as f:
        json.dump(all_chunks, f, ensure_ascii=False, indent=2)
    
    logger.info(f"Todos los chunks ({len(all_chunks)}) guardados en {all_chunks_path}")


# Definir rutas
current_dir = pathlib.Path(os.getcwd())
input_dir = os.path.join(current_dir, "chunks", "raw_text")  # Donde se guardaron los textos extraídos
output_dir = os.path.join(current_dir, "chunks", "processed")  # Donde guardar los chunks procesados

# Imprimir las rutas para verificar
print(f"Directorio de entrada: {input_dir}")
print(f"Directorio de salida: {output_dir}")

# Preguntar al usuario si las rutas son correctas antes de continuar
confirmation = input("¿Son correctas estas rutas? (s/n): ")
if confirmation.lower() != 's':
    print("Proceso cancelado.")
else:
    # Cargar textos
    texts = load_text_files(input_dir)
    print(f"Se cargaron {len(texts)} archivos de texto")
    
    # Procesar textos
    chunk_size = 1000  # Tamaño de chunk en caracteres (ajustar según necesidad)
    chunk_overlap = 200  # Superposición entre chunks (ajustar según necesidad)
    chunks_dict = process_all_texts(texts, chunk_size, chunk_overlap)
    
    # Guardar chunks procesados
    save_chunks(chunks_dict, output_dir)
    
    # Mostrar ejemplo de un chunk
    if chunks_dict:
        first_file = list(chunks_dict.keys())[0]
        if chunks_dict[first_file]:
            first_chunk = chunks_dict[first_file][0]
            print("\nEjemplo de chunk procesado:")
            print(f"Documento: {first_chunk['metadata']['source']}")
            print(f"Chunk ID: {first_chunk['metadata']['chunk_id']}")
            print(f"Texto: {first_chunk['text'][:300]}...")